<a href="https://colab.research.google.com/github/ageraustine/color-recognition-efficientNET/blob/master/Color_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Seeding for Randomization

In [1]:
import numpy as np
np.random.seed(345)
from tensorflow.random import set_seed
set_seed(3)

Essential Modules

In [2]:
import shutil
from collections import defaultdict
import json
from pathlib import Path
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from numpy import asarray
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, callbacks
from keras_preprocessing.image import ImageDataGenerator
tf.keras.backend.clear_session()
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn import datasets
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
import time
from datetime import datetime
import zipfile

Fetching the Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
vcor_zip_loc = '/content/drive/MyDrive/datasets/vcor/vcor_zipfile.zip'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Unzip the vcor zip file from drive

In [4]:
vcor_path = os.path.join(os.getcwd(),"datasets")
with zipfile.ZipFile(vcor_zip_loc, "r") as zp:
  zp.extractall(vcor_path)

Datasets directories

In [5]:
train_path = f"{vcor_path}/train/"
test_path = f"{vcor_path}/test/"
val_path = f"{vcor_path}/val/"

                         Data Preprocessing
1. Load the dataset                        

In [6]:
batch_size = 32
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  seed=123,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  val_path,
  seed=123,
  batch_size=batch_size)

Found 7267 files belonging to 15 classes.
Found 1550 files belonging to 15 classes.


2. Dataset Normalization

In [7]:
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) 
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

Model Building

In [8]:
# MODEL PARAMETERS
ISIZE = 256
NUM_CLASSES = 15

# Model layers
model_name='EfficientNetB3'
base_model= tf.keras.applications.EfficientNetB3(include_top=False, weights='imagenet',input_shape=(ISIZE, ISIZE, 3), pooling='max')
x= base_model.output
x= layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = layers.Dense(256, activation='relu')(x)
x= keras.layers.Dropout(rate=.45, seed=123)(x)

# softmax activation function for multi-class classification     
output= layers.Dense(NUM_CLASSES, activation='softmax')(x)
# Model Initialization
model= keras.Model(inputs=base_model.input, outputs=output)
#Model Compilation
model.compile(optimizers.Adamax(learning_rate=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
EPOCHS = 10
csv_logger = callbacks.CSVLogger('training.log', separator=',', append=False)

checkpointer = callbacks.ModelCheckpoint(filepath='model.h5', 
                               verbose=1, 
                               save_best_only=True)

early_stop = callbacks.EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')
history = model.fit(train_ds,
                    epochs=EPOCHS, 
                    validation_data=val_ds,
                    callbacks=[early_stop, checkpointer, csv_logger],
                    verbose=True,
                    shuffle = True,
                    workers=4)

Epoch 1/10
 52/228 [=====>........................] - ETA: 1:13:10 - loss: 3.4445 - accuracy: 0.1058